# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [7]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,blackmans bay,-43.0167,147.3167,65.52,51,34,9.44,AU,1737866183
1,1,ushuaia,-54.8000,-68.3000,47.86,87,20,10.36,AR,1737866184
2,2,lihue,21.9789,-159.3672,74.79,73,40,10.36,US,1737866185
3,3,ancud,-41.8697,-73.8203,55.13,93,14,4.07,CL,1737866186
4,4,georgetown,5.4112,100.3354,87.75,70,20,5.75,MY,1737865929


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [9]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    size="Humidity",
    color="City",
    alpha=0.7,
    hover_cols=["City", "Country", "Max Temp", "Humidity", "Cloudiness", "Wind Speed"],
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Max Temp,Cloudiness,Wind Speed)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [12]:
# Narrow down cities that fit criteria and drop any results with null values
humidity_threshold = 100
cloudiness_threshold = 100

# Drop any rows with null values
filtered_df = city_data_df[
    (city_data_df["Humidity"] <= humidity_threshold) &
    (city_data_df["Cloudiness"] <= cloudiness_threshold)
].dropna()

# Display sample data
filtered_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,blackmans bay,-43.0167,147.3167,65.52,51,34,9.44,AU,1737866183
1,1,ushuaia,-54.8000,-68.3000,47.86,87,20,10.36,AR,1737866184
2,2,lihue,21.9789,-159.3672,74.79,73,40,10.36,US,1737866185
3,3,ancud,-41.8697,-73.8203,55.13,93,14,4.07,CL,1737866186
4,4,georgetown,5.4112,100.3354,87.75,70,20,5.75,MY,1737865929


### Step 3: Create a new DataFrame called `hotel_df`.

In [13]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,blackmans bay,AU,-43.0167,147.3167,51,
1,ushuaia,AR,-54.8000,-68.3000,87,
2,lihue,US,21.9789,-159.3672,73,
3,ancud,CL,-41.8697,-73.8203,93,
4,georgetown,MY,5.4112,100.3354,70,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [14]:
# Set parameters to search for a hotel
radius = 10000 
params = {
    "categories": "accommodation.hotel",
    "limit": 1,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
blackmans bay - nearest hotel: Villa Howden
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
lihue - nearest hotel: Kauai Palms
ancud - nearest hotel: Cabañas
georgetown - nearest hotel: Page 63 hostel
santa catalina - nearest hotel: C&L Top View Hotel
cestos city - nearest hotel: No hotel found
sao joao do piaui - nearest hotel: Hotel Esplanada
puerto natales - nearest hotel: Dorotea Patagonia Hostel
callao - nearest hotel: Casa Ronald
piti village - nearest hotel: The Cliff Hotel
yellowknife - nearest hotel: No hotel found
muynoq - nearest hotel: Lala
midvagur - nearest hotel: GiljaHome
bethel - nearest hotel: Hampton Inn Danbury
el calafate - nearest hotel: Hostería Las Avutardas
port-aux-francais - nearest hotel: Keravel
iqaluit - nearest hotel: Frobisher Inn
anadyr - nearest hotel: Гостевой дом
college - nearest hotel: BEST WESTERN PLUS Pioneer Park Inn
iskateley - nearest hotel: Авантаж
avarua - nearest hotel: Paradise Inn
ilulissat - nearest hotel: Bes

,City,Country,Lat,Lng,Humidity,Hotel Name
0,blackmans bay,AU,-43.0167,147.3167,51,Villa Howden
1,ushuaia,AR,-54.8000,-68.3000,87,Apart Hotel Aires del Beagle
2,lihue,US,21.9789,-159.3672,73,Kauai Palms
3,ancud,CL,-41.8697,-73.8203,93,Cabañas
4,georgetown,MY,5.4112,100.3354,70,Page 63 hostel
...,...,...,...,...,...,...
595,papao,PF,-17.7500,-149.5500,76,Hiti Moana Villa Lodge
596,oroqen zizhiqi,CN,50.5667,123.7167,85,大鮮卑酒店
597,benguela,AO,-12.5763,13.4055,79,Hotel Moibela
598,yendi,GH,9.4427,-0.0099,31,White House Lodge


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [15]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points( 
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    size="Humidity",
    color="City",
    alpha=0.7,
    hover_cols=["City", "Country", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Hotel Name"],
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)